## Practica método de Muestreo aleatorio para la inferencia en Redes Bayesianas

En esta práctica vamos a programar el método de muestreo aleatorio para la inferencia en Redes Bayesianas. 

En el método del muestreo aleatorio se generan N ejemplos de la distribución de la RB. Estos ejemplos se generan muestreando cada nodo de la red de arriba a abajo. Después,se utilizan las muestras que contienen la evidencia = e y la querie para estimar la probabilidad condicionada.

## Nodos y Grafos 
Para implementar los nodos de una Red Bayesiana y el Grafo vamos a utilizar clases y diccionarios de python. En la siguiente celda la clase nodo representa un nodo de la Red Bayesiana. Como información tiene:
- padres: lista donde se almacenan los nodos padres
- hijos: lista donde se almacenan los nodos hijos
- p: lista con la tabla de probabilidades
- state: El estado de la variable. En principio como es una variable aletaria desconocemos su valor, por eso se inicia a None. Si 'vemos' el valor de la variable, es decir si esa variable es una evidencia, conocemos su valor. Entonces podremos cambiar el estado al valor de la variable. En esta práctica las variables son binarias, por lo tanto el estado podrá ser 0 (positivo) o 1 (negativo).
- num_states: el número de posibles estados
- states: los valores de los estados (como vamos a trabajar con variables binarias los estados serán 0 o 1.


In [1]:
import random
class nodo:
    def __init__(self, padres, hijos, p):
        self.padres = padres
        self.hijos = hijos
        self.p = p
        
        self.num_states = 2
        self.states = [0, 1]
        self.state = None
        
    
    def sample(self, estado_padres):
        self.state = None
        
            

En esta práctica vamos a trabajar con la RB que tenía las variables Nublado, Aspersor, Lluvia y Mojado. En la siguiente celda definimos la distribución de probabilidad de la variable Nublado y creamos un objeto nodo. Las listas de hijos y padres continen la información de las relaciones entre las variables. Vamos a crear unas etiquetas, valores enteros, para identificar cada nodo:
- Nublado: 0
- Aspersor: 1
- Lluvia: 2
- Mojado: 3

In [2]:
# Nublado
pN = [[0.5, 0.5]]
Nublado = nodo([], [1,2], pN)

In [3]:
# Aspersor
pA = [[0.1, 0.9], [0.5, 0.5]]
Aspersor = nodo([0], [3], pA)

Si quiero ver la probabilidad de que el aspersor se encienda si el cielo está nublado:

In [4]:
# pongo el estado de nublado a positivo:
Nublado.state = 0
print(Aspersor.p[Nublado.state][0])

0.1


Completa la tabla con la distribución de la variable Lluvia y la variable Mojado y crea sus nodos:

In [5]:
# Lluvia
pLl = [[0.8, 0.2], [0.3, 0.7]]
Lluvia = nodo([0], [3], pLl)

In [6]:
#Mojado
pM = [[1, 0], [0.9, 0.1], [0.9, 0.1], [0.01, 0.99]]
Mojado = nodo([1,2], [], pM)

Pon los estados de aspersor y lluvia con valor 1 y comprueba la probabilidad de que el suelo esté mojado condicionado a no aspersor y no lluvia: 

In [7]:
Aspersor.state = 1
Lluvia.state = 1
print(Mojado.p[Aspersor.state][Lluvia.state])

0.1


El Grafo lo guardamos en un diccionario en el que la clave es la etiqueta de cada nodo y el valor es el propio nodo:

In [8]:
Grafo = {0: Nublado, 1:Aspersor, 2:Lluvia, 3:Mojado}

In [9]:
# Esta función pasa los estados de cada nodo del grafo a un diccionario y lo imprime
def imprime(g):
    totalstate={}
    for nodo in g.keys():
        totalstate[nodo] = g[nodo].state
    print(totalstate)
imprime(Grafo)

{0: 0, 1: 1, 2: 1, 3: None}


Crea una función que recorra todos los nodos de un grafo y ponga todos sus estados a None:

In [10]:
def reset_grafo(g):
    for nodo in g.values():
        nodo.state = None

    return g

Ahora vamos implementar un recorrido en anchura del grafo para generar una muestra. El objetivo es ir recorriendo en anhura los nodos del grafo, y en cada nodo mediante un número aleatorio modificar el estado del nodo. Para modificar el estado del nodo, primero debes volver a la celda inicial y completar el método sample(). Este nuevo nodo tiene como parámetro una lista con los estados de los nodos padres para poder acceder a la probabilidad condicionada y cambia el estado del nodo a 0 o 1 dependiendo del valor aleatorio que se genera dentro del método. Ten en cuenta que en nuestro ejemplo hay un nodo que no tiene ningún padre, otros nodos tiene un padre y otro nodo tiene dos padres. 
Comprueba que el método sample funciona:

In [11]:
Aspersor.sample([0])
print(Aspersor.state)

None


Completa la función recorrido_anchura en que recorras el grafo en anchura. Utiliza la versión iterativa con la que trabajamos en búsquedas con la lista de abiertos y cerrados. En este caso cada vez que tengas un nodo en actual se llama al método sample y se cambia el estado de ese nodo. Al acabar de recorrer el grafo, todas las variables tendrán un estado, es decir se habrá generado una muestra aleatoria. El nodo que se pasa como parámetro es el nodo raiz del grafo.

In [13]:
from collections import deque

def recorrido_anchura(node, graph):
    abiertos = deque()
    cerrados = []

    # Agregamos el nodo raíz a la lista de nodos abiertos
    abiertos.append(node)

    while abiertos:
        actual = abiertos.popleft()
        cerrados.append(actual)
        estados = []

        # Obtener los estados de los nodos padres
        for i in actual.padres:
            estados.append(graph[i].state)

        # Llamar al método sample y cambiar el estado del nodo actual
        actual.sample(estados)

        # Agregar los nodos hijos a la lista de nodos abiertos si no están en abiertos ni cerrados
        for hijo in actual.hijos:
            if hijo not in abiertos and hijo not in cerrados:
                abiertos.append(graph[hijo])

    return graph


Cada muestra la vamos a representar mediante un diccionario, por ejemplo este diccionario es una muestra:

{0:1, 1:0, 2:1, 3:0}

Crea una función que primero resetea el grafo, después realiza un recorrido en anchura y finalmente devuelve una muestra con los estados de los nodos del grafo.

In [14]:
def sample_grafo(node, graph):
    # Resetear el grafo
    reset_grafo(graph)

    # Realizar el recorrido en anchura
    recorrido_anchura(node, graph)

    # Crear y devolver una muestra con los estados de los nodos del grafo
    states = {nodo: graph[nodo].state for nodo in graph}
    return states

Finalmente vamos a a crear una función que realice la inferencia aproximada utilizando el método del muestreo con rechazo. La función tendrá como parámetros una query, una evidencia y un grafo. Las queries y las evidencias las vamos a representar mediante diccionarios, por ejemplo: Si queremos saber $P(+m|\neg n)$ entonces:
- query = {3:0} # El valor de la variable Mojado es 0.
- evidencia = {0:1} # El valor de la variable Nublado es 1.


Si nos preguntan $P(+Ll| +a, +m)$
- query = {2:0}
- evidence = {1:0, 3:0}


In [16]:
import random

def infer(query, evidence, graph, num=20000):
    n_evidence = 0
    n_query_and_evidence = 0

    # Realizar el muestreo
    for _ in range(num):
        # Generar muestras aleatorias de los estados de las variables en el grafo
        states = sample_grafo(list(graph.keys())[0], graph)  # Se asume que el nodo raíz es el primer nodo en el grafo

        # Comprobar si la muestra satisface la evidencia
        evidence_satisfied = all(states[key] == value for key, value in evidence.items())
        if evidence_satisfied:
            n_evidence += 1

            # Comprobar si la muestra satisface tanto la evidencia como la consulta
            query_and_evidence_satisfied = all(states[key] == value for key, value in query.items())
            if query_and_evidence_satisfied:
                n_query_and_evidence += 1

    # Calcular la probabilidad condicional aproximada
    if n_evidence == 0:
        return 0.0
    else:
        return n_query_and_evidence / n_evidence


Comprueba $P(+m|\neg n)$ y $P(+Ll| +a, +m)$ con los resultados que hemos obtenido utilizando el método de enumeración.